# Retomando conf inicial

In [1]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation
import pandas as pd
import numpy as np
import sys
sys.path.append('../code/')
from conf_inicial_2d import *
from fuerzas import *
from integradores import *
plt.style.use('seaborn-whitegrid')
%matplotlib inline

N = 5
dens = 1.0
iter_check = 40
elements=['He']

dic = conf_inicial_2d(N = N,\
                      dens=dens,\
                      elements=elements,\
                      iter_check=iter_check)

In [4]:
dic.keys()

dict_keys(['df', 'dr', 'box', 'path_out'])

In [5]:
df = dic['df']

In [6]:
df

,x,y,vx,vy,element,m,e,sizes,fx,fy
0,1.042208,2.215411,-0.977278,-0.103219,He,4.002602,0.817641,400.2602,32.430110,-181.868627
1,1.834085,1.403991,0.410599,0.761038,He,4.002602,0.804002,400.2602,-178.664625,-60.690239
2,0.222575,0.416173,0.121675,0.443863,He,4.002602,0.809787,400.2602,135.327128,101.745936
3,1.623684,0.356015,0.333674,1.494079,He,4.002602,0.822409,400.2602,-87.271757,158.670696
4,0.867618,1.198579,-0.205158,-2.552990,He,4.002602,0.818676,400.2602,98.179144,-17.857766


In [7]:
# ver codigo addvel.f

# Crear metodo para correr addve.f desde codigo py

### dar datos a programa 

In [10]:
# compilamos y lo ponemos en carpeta code

In [9]:
# Ejemplo de codigo usado en un proyecto

cmd = [gmx, 'mdrun',
       '-s', PATH_NVT2_TPR,
       '-o', PATH_NVT2_TRR,
       '-c', PATH_NVT2_CONFOUT,
       '-e', PATH_NVT2_EDR,
       '-g', PATH_NVT2_LOG,
       '-cpo', PATH_NVT2_CPT,
       '-nt', NUM_PROC]

cmd = [str(item) for item in cmd]


process = subprocess.run(cmd, check=True, stdout=subprocess.PIPE, universal_newlines=True)

f = open('tmpE.dat', 'w')
f.write("35\n0\n")
f.close()

cmd = [gmx, 'energy',
       '-f', PATH_NVT2_EDR,
       '-o', PATH_NVT2_SURFT_XVG_DAT,
       '-xvg', 'none',
       '<', 'tmpE.dat']
cmd = [str(item) for item in cmd]
cmd = ' '.join(cmd)
process = subprocess.run(cmd, check=True, shell=True, stdout=subprocess.PIPE, universal_newlines=True)   


In [12]:
# azar.f pregunta lo siguiente

In [ ]:
#  enter file
# azar.dat
#  file       azar.dat            
#  Enter file to keep last data
# azar2.dat
#  Enter number of data
# 10
#  enter number of NON-frozen particles
# 10
#  TEMPERATURE
# 1
#  TEMP0=  0.97716428482344941     
#  PX0,PY0,PZ0=  -2.2204460492503131E-016  -2.2204460492503131E-016   4.4408920985006262E-016
#  ia=          10
#  TEMP=  0.97716428482344941     
#  PX,PY,PZ=  -2.2204460492503131E-016  -2.2204460492503131E-016   4.4408920985006262E-016


In [11]:
# por lo pronto el azar.dat lo tomamos de fortran

In [14]:
import sys
sys.path.append('../code/')
import os

PATH_VEL_BIN = "../bin/addvel.exe"


COORD_FILE = "azar.dat"
ADDVEL_FILE = "azar2.dat" #otros nombres sug

NUM_PARTICULAS = 10
NON_FROZEN = 10
TEMPERATURA = 1.0

f = open('tmpE.dat', 'w')
f.write("{}\n{}\n{}\n{}\n{}".format(COORD_FILE,
                                    ADDVEL_FILE,
                                    NUM_PARTICULAS,
                                    NON_FROZEN,
                                    TEMPERATURA))
f.close() # lo vemos _escribir

In [15]:
# continuamos con ejecucion

In [18]:
import sys
sys.path.append('../code/')
import os
import subprocess # nueva libreria propia py

PATH_BIN = "/home/antadlp/Documents/grupo-hd_curso-python3/bin"

NAME_VEL_BIN = "addvel.exe"
NAME_INP_FILE = "tmp.inp"

PATH_VEL_BIN = os.path.join(PATH_BIN, NAME_VEL_BIN) # ver ejemplos, salidas
PATH_INP_FILE = os.path.join(PATH_BIN, NAME_INP_FILE) 

COORD_FILE = "azar.dat"
ADDVEL_FILE = "azar2.dat" #otros nombres sug
NUM_PARTICULAS = 10
NON_FROZEN = 10
TEMPERATURA = 1.0

f = open(PATH_INP_FILE, 'w')
f.write("{}\n{}\n{}\n{}\n{}".format(COORD_FILE,
                                    ADDVEL_FILE,
                                    NUM_PARTICULAS,
                                    NON_FROZEN,
                                    TEMPERATURA))

cmd = [PATH_VEL_BIN, '<', NAME_INP_FILE] # el programa externo busca el input en su mismo nivel
cmd = [str(item) for item in cmd]
cmd = ' '.join(cmd)
process = subprocess.run(cmd, check=True, shell=True, stdout=subprocess.PIPE, universal_newlines=True) 

# _escribir y probar


# Hacer de conf2d a formato md.org y xyz

In [19]:
# antes quitar la parte de velocidades y agregar dim z

In [ ]:
import pandas as pd
import numpy as np
from mendeleev import element
from fuerzas import *

def ranf():
    return np.random.uniform()

def conf_inicial_2d(**kwargs):
    
    N = kwargs.get('N', 20)
    dens = kwargs.get('dens', 1)
    iter_check = kwargs.get('iter_check', 10)
    tol = kwargs.get('tol', 0.7)
    seed = kwargs.get('seed', 0)
    path_out = kwargs.get('path_out', 'conf_inicial.pkl')
    elements = kwargs.get('elements', ['He'])
    fsize = kwargs.get('fsize', 250)
   
    np.random.seed(seed)
    elms = np.random.choice(elements, size=N)
    
    mu_e =  kwargs.get('mu_e', 0.7)
    sigma_e = kwargs.get('sigma_e', 0.3)
    dist_e = np.abs(np.random.normal(mu_e, sigma_e, N))
    charges = kwargs.get('charges', dist_e)

    box = np.power(np.divide(N, dens), np.divide(1, 2))
    box2 = box/2

    r = {}
    r['x'] = {}
    r['y'] = {}
    r['vx'] = {}
    r['vy'] = {}
    r['element'] = {}
    r['m'] = {}
    r['e'] = {}
    r['sizes'] = {}
    
    for i in range(N):
        r['x'][i] = box*ranf() - box2
        r['y'][i] = box*ranf() - box2
        r['vx'][i] = np.random.randn()
        r['vy'][i] = np.random.randn()
        el = elms[i]
        m = element(el).atomic_weight        
        r['element'][i] = el
        r['m'][i] = m
        r['e'][i] = charges[i]
        r['sizes'][i] = fsize*m


    for k in range(iter_check):
        dr = 0
        for i in range(N - 1):
            rxi = r['x'][i]
            ryi = r['y'][i]
            jlist = np.arange(i+1, N, 1)
            for j in jlist:
                rxij = rxi - r['x'][j]
                ryij = ryi - r['y'][j]
                rsij = rxij*rxij+ryij*ryij
                flag_rsij = False
                if (rsij < tol):
                    flag_rsij = True
                while(flag_rsij):
                    r['x'][j] = box*ranf() - box2
                    r['y'][j] = box*ranf() - box2
                    dr=dr+1
                    rxij = rxi - r['x'][j]
                    ryij = ryi - r['y'][j]
                    rsij = rxij*rxij+ryij*ryij

                    if (rsij >= 0.7):
                        flag_rsij = False
            
    dfr = pd.DataFrame(r)

    center = dfr[['x', 'y']].mean().values
    mv = np.array([box/2, box/2]) - center
    rdic = {}
    rdic['x'] = {}
    rdic['y'] = {}
    for idx in dfr.index:
        rdic['x'][idx] = dfr['x'].loc[idx] + mv[0]
        rdic['y'][idx] = dfr['y'].loc[idx] + mv[1]

    dfr.update(pd.DataFrame(rdic))
    dfr = dfr.join(get_coulomb2d(dfr))
   
    dic = {}
    dic['df'] = dfr
    dic['dr'] = dr
    dic['box'] = box
    dic['path_out'] = path_out
    
    dfr.to_pickle(path_out)
    
    return dic

In [21]:
# copiar y hacerlo directo en consola, juntos

import pandas as pd
import numpy as np
from mendeleev import element
from fuerzas import *

def ranf():
    return np.random.uniform()

def conf_inicial_md(**kwargs):
    
    N = kwargs.get('N', 20)
    dens = kwargs.get('dens', 1)
    iter_check = kwargs.get('iter_check', 10)
    tol = kwargs.get('tol', 0.7)
    seed = kwargs.get('seed', 0)
    path_out = kwargs.get('path_out', 'conf_inicial.pkl')
    elements = kwargs.get('elements', ['He'])
   
    np.random.seed(seed)
    elms = np.random.choice(elements, size=N)
    
    mu_e =  kwargs.get('mu_e', 0.7)
    sigma_e = kwargs.get('sigma_e', 0.3)
    dist_e = np.abs(np.random.normal(mu_e, sigma_e, N))
    charges = kwargs.get('charges', dist_e)

    box = np.power(np.divide(N, dens), np.divide(1, 2))
    box2 = box/2

    r = {}
    r['x'] = {}
    r['y'] = {}
    r['z'] = {}
    r['vx'] = {}
    r['vy'] = {}
    r['vz'] = {}
    r['element'] = {}
    r['m'] = {}
    r['e'] = {}
    
    for i in range(N):
        r['x'][i] = box*ranf() - box2
        r['y'][i] = box*ranf() - box2
        r['z'][i] = box*ranf() - box2
        el = elms[i]
        m = element(el).atomic_weight        
        r['element'][i] = el
        r['m'][i] = m
        r['e'][i] = charges[i]

    for k in range(iter_check):
        dr = 0
        for i in range(N - 1):
            rxi = r['x'][i]
            ryi = r['y'][i]
            rzi = r['z'][i]
            jlist = np.arange(i+1, N, 1)
            for j in jlist:
                rxij = rxi - r['x'][j]
                ryij = ryi - r['y'][j]
                rzij = ryi - r['y'][j]
                rsij = rxij*rxij+ryij*ryij+rzij*rzij
                flag_rsij = False
                if (rsij < tol):
                    flag_rsij = True
                while(flag_rsij):
                    r['x'][j] = box*ranf() - box2
                    r['y'][j] = box*ranf() - box2
                    r['z'][j] = box*ranf() - box2
                    dr=dr+1
                    rxij = rxi - r['x'][j]
                    ryij = ryi - r['y'][j]
                    rzij = rzi - r['z'][j]
                    rsij = rxij*rxij+ryij*ryij+rzij*rzij                    

                    if (rsij >= 0.7):
                        flag_rsij = False
            
    dfr = pd.DataFrame(r)
   
    dic = {}
    dic['df'] = dfr
    dic['dr'] = dr
    dic['box'] = box
    dic['path_out'] = path_out
    
    dfr.to_pickle(path_out)
    
    return dic

In [22]:
# prueba sencilla

In [23]:
N = 5
dens = 1.0
iter_check = 40
elements=['He']
fsize=100
mu_e = 0.8
sigma_e = 0.01

dic = conf_inicial_md(N = N,
                      dens=dens,
                      elements=elements,
                      iter_check=iter_check,
                      fsize=fsize,
                      mu_e=mu_e,
                      sigma_e=sigma_e)

# _escribimos y salidas en csv, excel y pkl

In [24]:
df = dic['df']

In [25]:
df

,x,y,z,vx,vy,vz,element,m,e
0,-0.139559,0.876031,1.036781,NaN,NaN,NaN,He,4.002602,0.817641
1,0.668939,-0.086135,0.627282,NaN,NaN,NaN,He,4.002602,0.804002
2,-0.853564,0.312873,-0.797486,NaN,NaN,NaN,He,4.002602,0.809787
3,-0.923207,-1.072824,0.743761,NaN,NaN,NaN,He,4.002602,0.822409
4,0.153023,-1.076019,0.263041,NaN,NaN,NaN,He,4.002602,0.818676


In [27]:
# lo ponemos en carpeta codigo, crear archivo coordenadas para darselo a addvel.exe

In [ ]:
# muestra archvo

#     10
#    2.154435  2.154435  2.154435
#      -1.03169     0.57403     0.41201
#      -0.41703    -0.34806    -0.47661
#       0.82259    -0.20722     0.10347
#      -0.89688    -0.75069     1.03016
#       0.43390     0.55711     0.22849
#       0.05917     0.60859    -0.66077
#       0.91802     1.04208    -0.56146
#       0.17272    -1.04544    -0.64979
#       0.84889     1.00572     0.80313
#       0.13780    -0.34899     0.72431

In [2]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../code/')
from conf_inicial_md import *
import os

PATH_GRL = "/home/antadlp/Documents/grupo-hd_curso-python3/notebooks"
NAME_COORD = "azar.dat"
PATH_COORD = os.path.join(PATH_GRL, NAME_COORD)

N = 10
dens = 1.0
iter_check = 40
elements=['He']

dic = conf_inicial_md(N = N,
                      dens=dens,
                      elements=elements,
                      iter_check=iter_check,
                      fsize=fsize,
                      mu_e=mu_e,
                      sigma_e=sigma_e)

df = dic['df']
L = dic['box']
L = np.round(L, decimals=4)

f = open(PATH_COORD, 'w')
f.write("{}\n".format(len(df)))
f.write("{:14.5f}{:14.5f}{:14.5f}\n".format(L, L, L))

for idx in df.index:
    x = df['x'].loc[idx]
    y = df['y'].loc[idx]
    z = df['z'].loc[idx]
    
    f.write("{:15.5f}{:12.5f}{:12.5f}\n".format(x, y, z)) #salto linea
    
    
f.close()   #_escribir

In [3]:
# probando alimentando a addvel.f, juntos copy y paste del codigo anterior, modificamos juntos

In [ ]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../code/')
from conf_inicial_md import *
import os

PATH_GRL = "/home/antadlp/Documents/grupo-hd_curso-python3/notebooks"
NAME_COORD = "azar.dat"
PATH_COORD = os.path.join(PATH_GRL, NAME_COORD)

N = 10
dens = 1.0
iter_check = 40
elements=['He']

dic = conf_inicial_md(N = N,
                      dens=dens,
                      elements=elements,
                      iter_check=iter_check,
                      fsize=fsize,
                      mu_e=mu_e,
                      sigma_e=sigma_e)

df = dic['df']
L = dic['box']
L = np.round(L, decimals=4)

f = open(PATH_COORD, 'w')
f.write("{}\n".format(len(df)))
f.write("{:14.5f}{:14.5f}{:14.5f}\n".format(L, L, L))

for idx in df.index:
    x = df['x'].loc[idx]
    y = df['y'].loc[idx]
    z = df['z'].loc[idx]
    
    f.write("{:15.5f}{:12.5f}{:12.5f}\n".format(x, y, z)) #salto linea
    
    
f.close()   #_escribir



import sys
sys.path.append('../code/')
import os
import subprocess # nueva libreria propia py

PATH_BIN = "/home/antadlp/Documents/grupo-hd_curso-python3/bin"

NAME_VEL_BIN = "addvel.exe"
NAME_INP_FILE = "tmp.inp"

PATH_VEL_BIN = os.path.join(PATH_BIN, NAME_VEL_BIN) # ver ejemplos, salidas
PATH_INP_FILE = os.path.join(PATH_BIN, NAME_INP_FILE) 

COORD_FILE = "azar.dat"
ADDVEL_FILE = "azar2.dat" #otros nombres sug
NUM_PARTICULAS = 10
NON_FROZEN = 10
TEMPERATURA = 1.0

f = open(PATH_INP_FILE, 'w')
f.write("{}\n{}\n{}\n{}\n{}".format(COORD_FILE,
                                    ADDVEL_FILE,
                                    NUM_PARTICULAS,
                                    NON_FROZEN,
                                    TEMPERATURA))

cmd = [PATH_VEL_BIN, '<', NAME_INP_FILE] # el programa externo busca el input en su mismo nivel
cmd = [str(item) for item in cmd]
cmd = ' '.join(cmd)
process = subprocess.run(cmd, check=True, shell=True, stdout=subprocess.PIPE, universal_newlines=True) 

# _escribir y probar


In [ ]:
# modificando..

In [ ]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../code/')
from conf_inicial_md import *
import subprocess 
import os

PATH_BIN = "/home/antadlp/Documents/grupo-hd_curso-python3/bin"

NAME_COORD = "azarx.dat"
NAME_VEL_BIN = "addvel.exe"
NAME_INP_FILE = "tmp2.inp"
ADDVEL_FILE = "azar3.dat"


PATH_COORD = os.path.join(PATH_BIN, NAME_COORD)
PATH_VEL_BIN = os.path.join(PATH_BIN, NAME_VEL_BIN)
PATH_INP_FILE = os.path.join(PATH_BIN, NAME_INP_FILE) 

NUM_PARTICULAS = 10
dens = 1.0
iter_check = 40
elements=['He']
NON_FROZEN = 10
TEMPERATURA = 1.0

dic = conf_inicial_md(N = NUM_PARTICULAS,
                      dens=dens,
                      elements=elements,
                      iter_check=iter_check)

df = dic['df']
L = dic['box']
L = np.round(L, decimals=4)

f = open(NAME_COORD, 'w')
f.write("{}\n".format(len(df)))
f.write("{:14.5f}{:14.5f}{:14.5f}\n".format(L, L, L))

for idx in df.index:
    x = df['x'].loc[idx]
    y = df['y'].loc[idx]
    z = df['z'].loc[idx]
    
    f.write("{:15.5f}{:12.5f}{:12.5f}\n".format(x, y, z)) #salto linea
    
    
f.close()   


f = open(PATH_INP_FILE, 'w')
f.write("{}\n{}\n{}\n{}\n{}".format(NAME_COORD,
                                    ADDVEL_FILE,
                                    NUM_PARTICULAS,
                                    NON_FROZEN,
                                    TEMPERATURA))

f.close()
cmd = [PATH_VEL_BIN, '<', NAME_INP_FILE] # el programa externo busca el input en su mismo nivel
cmd = [str(item) for item in cmd]
cmd = ' '.join(cmd)
process = subprocess.run(cmd, check=True, shell=True, stdout=subprocess.PIPE, universal_newlines=True) 

# _probar

In [3]:
PATH_VEL_BIN

'/home/antadlp/Documents/grupo-hd_curso-python3/bin/addvel.exe'

In [4]:
NAME_INP_FILE

'tmp2.inp'